Получение метрик по моделям триплетов, создание файлов KG триплетов моделями до и послеобучения

In [1]:
from typing import List, Dict, Set
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text
import os
import json
import re
import numpy as np
from tqdm import tqdm
import pandas as pd

In [2]:
# Параметры скрипта

# DOMAIN = 'movie'
# DOMAIN = 'computer'
DOMAIN = 'nature'


if DOMAIN == 'movie':
    ONTOLOGY_PATH = 'data/onotology_schema/1_movie_ontology.json'
    SCHEMA_PATH = 'data/onotology_schema/movie_schema.json'
    GROUND_TRUTH_PATH = 'data/movie_ground_truth.jsonl'

elif DOMAIN == 'computer':
    ONTOLOGY_PATH = 'data/onotology_schema/6_computer_ontology.json'
    SCHEMA_PATH = 'data/onotology_schema/computer_schema.json'
    GROUND_TRUTH_PATH = 'data/computer_ground_truth.jsonl'

elif DOMAIN == 'nature':
    ONTOLOGY_PATH = 'data/onotology_schema/9_nature_ontology.json'
    SCHEMA_PATH = 'data/onotology_schema/nature_schema.json'
    GROUND_TRUTH_PATH = 'data/nature_ground_truth.jsonl'




# Подготовка
## Загрузка онтологии

In [3]:
with open(ONTOLOGY_PATH, 'r') as f:
    ontology_filejson = json.load(f)

In [4]:
ont_str = '\n'.join(ontology_filejson['payload']['blob']['rawLines'])
ont = json.loads(ont_str)

if DOMAIN == 'movie':
    # Неиспользуемая сущность в онтологии
    ont['concepts'] = [x for x in ont['concepts'] if x['qid']
                       not in ['Q201658', 'Q4220917', 'Q104649845']]

if DOMAIN == 'nature':
    # Неиспользуемая сущность в онтологии
    ont['concepts'] = [x for x in ont['concepts'] if x['qid']
                       not in ['Q82673', 'Q15091377']]

ont

{'title': 'Nature Ontology',
 'id': 'ont_9_nature',
 'concepts': [{'qid': 'Q618123', 'label': 'geographical feature'},
  {'qid': 'Q131681', 'label': 'reservoir'},
  {'qid': 'Q52105', 'label': 'Habitat'},
  {'qid': 'Q12323', 'label': 'dam'},
  {'qid': 'Q166620', 'label': 'drainage basin'},
  {'qid': 'Q46831', 'label': 'mountain range'},
  {'qid': 'Q8502', 'label': 'mountain'},
  {'qid': 'Q16521', 'label': 'taxon'},
  {'qid': 'Q355304', 'label': 'watercourse'},
  {'qid': 'Q15324', 'label': 'body of water'},
  {'qid': 'Q5', 'label': 'human'},
  {'qid': 'Q1040689', 'label': 'synonym'}],
 'relations': [{'pid': 'P171',
   'label': 'parent taxon',
   'domain': 'Q16521',
   'range': 'Q16521'},
  {'pid': 'P4552',
   'label': 'mountain range',
   'domain': 'Q8502',
   'range': 'Q46831'},
  {'pid': 'P3137',
   'label': 'parent peak',
   'domain': 'Q8502',
   'range': 'Q8502'},
  {'pid': 'P1843',
   'label': 'taxon common name',
   'domain': 'Q7432',
   'range': ''},
  {'pid': 'P974',
   'label': 

In [5]:
concepts_dict = {x['qid']: x['label'] for x in ont['concepts']}
relations_with_text = [{**relation, 'domain_label': concepts_dict.get(
    relation['domain']), 'range_label': concepts_dict.get(relation['range'])} for relation in ont['relations']]

# ручная коррекция пропущенного значения
if DOMAIN == 'computer':
    relations_with_text = [dict(x, range_label='company') if x['label']
                           == 'developer' else x for x in relations_with_text]

# удаляем None в obj
relations_with_text = [x for x in relations_with_text if x['range_label'] is not None]

concepts_list = [x['label'] for x in ont['concepts']]
concepts = ', '.join(concepts_list)

relations_list = []
for i in relations_with_text:
    rel = f"sub: {i['domain_label']}, rel: {i['label']}, obj: {i['range_label']}\n"
    relations_list.append(rel)
relations_schema = "".join(relations_list)
print(relations_schema)

sub: taxon, rel: parent taxon, obj: taxon
sub: mountain, rel: mountain range, obj: mountain range
sub: mountain, rel: parent peak, obj: mountain
sub: watercourse, rel: tributary, obj: watercourse
sub: watercourse, rel: origin of the watercourse, obj: body of water
sub: watercourse, rel: mouth of the watercourse, obj: watercourse
sub: taxon, rel: taxon synonym, obj: synonym
sub: dam, rel: reservoir created, obj: reservoir
sub: watercourse, rel: drainage basin, obj: drainage basin
sub: None, rel: mountains classification , obj: human
sub: taxon, rel: Habitat, obj: body of water



In [6]:
with open(SCHEMA_PATH, 'r') as f:
    sch = json.load(f)

schema = Object(
    id=sch['id'],
    description=sch['description'],
    attributes=[Text(id=x['id'], description=x['description'], examples=[tuple(x['examples'])]) 
                for x in sch['attributes']],
    examples = [tuple(x) for x in sch['examples']],
    many=True
)

## Генератор триплетов

In [7]:
GET_RELATIONS_TEMPLATE = """You are a networked intelligence helping a human track knowledge triples.
You are given a list of NAMED ENTITIES and a LIST OF POSIBLE RELATIONS. You are to identify the relations between the named entities. 
Do not use any other named entities or relations. You MUST use only provided NAMED ENTITIES and LIST OF POSIBLE RELATIONS. 

LIST OF POSIBLE RELATIONS: {relations}

NAMED ENTITIES: {entities}

To indetidy the relations between named entities, you can use the following context: {context}

Format your response as a JSON. Do not add any attributes that do not appear in the schema shown below:

    {{
    "triplets": [
        {{
        "sub": "string",
        "rel": "string",
        "obj": "string"
        }}
    ]
    }}

Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.
Answer:"""

In [8]:
relation_prompt = PromptTemplate.from_template(GET_RELATIONS_TEMPLATE)

In [9]:
class TripletBuilder:
    models_dict = {
        1: "../models/zephyr-7b-beta.Q6_K.gguf",
        2: "../models/mistral-7b-v0.1.Q5_K_M.gguf",
        3: "../models/llama-2-13b-chat.Q5_K_M.gguf",
        4: "../models/openchat-3.5-0106.Q8_0.gguf",
        5: "../models/mixtral-8x7b-v0.1.Q6_K.gguf",
        6: "../models/llama-2-7b.Q4_0.gguf"
    }

    def __init__(self, relations: str, models: int = 6, n_gpu_layers: int = 50,
                 model1_path: str = None, model2_path: str = None,
                 schema=schema, relation_prompt=relation_prompt, n_runs: int = 50):
        """
        Initialize the TripletBuilder class.

        Args:
            relations (str): The relations to extract triplets for.
            models (int, optional): The number of models to choose from. Defaults to 5.
            n_gpu_layers (int, optional): The number of GPU layers. Defaults to 50.
            model1_path (str, optional): The path to the model for first stage. Defaults to None.
            model2_path (str, optional): The path to the model for second stage. Defaults to None.
            schema (str, optional): The schema for named entity recognition. Defaults to schema.
            relation_prompt (str, optional): The prompt for relation extraction. Defaults to relation_prompt.
            n_runs (int, optional): The number of runs to perform. Defaults to 50.
        """
        if model1_path:
            self.model1_path = model1_path
        else:
            self.model1_path = self.models_dict[models]

        if model2_path:
            self.model2_path = model2_path
        else:
            self.model2_path = self.models_dict[models]

        self.schema = schema
        self.relations = relations
        self.relation_prompt = relation_prompt
        self.n_runs = n_runs

        self.llm1 = LlamaCpp(
            model_path=self.model1_path,
            temperature=0,
            max_tokens=256,
            n_gpu_layers=n_gpu_layers,
            n_batch=16,
            n_ctx=2048,
        )

        if self.model2_path == self.model1_path:
            self.llm2 = self.llm1
        else:
            self.llm2 = LlamaCpp(
                model_path=self.model2_path,
                temperature=0,
                max_tokens=256,
                n_gpu_layers=n_gpu_layers,
                n_batch=16,
                n_ctx=2048,
            )

        self.ner_chain = create_extraction_chain(
            self.llm1, self.schema, encoder_or_encoder_class='json')

        self.relations_chain = self.relation_prompt | self.llm2 | JsonOutputParser()

    def load_data(self, file_path: str, mask_index=None):
        """
        Load data from a file.

        Args:
            file_path (str): The path to the file.
        """
        with open(file_path, 'r') as f:
            self.data = f.readlines()

        if mask_index is not None:
            self.data = [self.data[i] for i in mask_index]

    def get_triplets(self):
        """
        Extract triplets from the data and return a list of dictionaries containing the model name, sentence, and triplets.

        Returns:
            list: A list of dictionaries, each containing the model name, sentence, and triplets.
        """
        result = []

        for i_text in tqdm(self.data[:self.n_runs]):
            i_text = json.loads(i_text)
            temp_dict = dict()

            context = i_text['sent']
            try:
                entities = self.ner_chain.invoke(
                    context)['text']['data'][DOMAIN]
            except:
                entities = []

            print(context)
            print("++++++++++++++++++++++++++++++++++++")
            print(entities)
            print("++++++++++++++++++++++++++++++++++++")

            try:
                if len(entities[0]) < 2:
                    triplets = {"triplets": []}
                else:
                    try:
                        triplets = self.relations_chain.invoke(
                            {"relations": self.relations, "entities": entities, "context": context})
                    except:
                        triplets = {"triplets": []}
            except:
                triplets = {"triplets": []}

            print(triplets)

            temp_dict['model1'] = os.path.basename(self.model1_path)
            temp_dict['model2'] = os.path.basename(self.model2_path)
            temp_dict['sent'] = context
            temp_dict['triples'] = triplets['triplets'] if isinstance(
                triplets, dict) and 'triplets' in triplets.keys() else triplets

            print("====================================")
            print(temp_dict)
            print("====================================")

            result.append(temp_dict)

        return result

# Извлечение триплетов
В результате будет три набора триплетов по тестовой части: ground truth, base, fine tuned

In [10]:
# номера строк, используемые в тесте
test_index = pd.read_csv(os.path.join('artifacts', DOMAIN, 'model_1/Xy_test.csv'), index_col='Unnamed: 0').index

In [11]:
builder = TripletBuilder(relations_schema,
                         n_runs=1000)

builder.load_data(GROUND_TRUTH_PATH, test_index[:3])

builder.get_triplets()


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../models/llama-2-7b.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32             

Chapleau River is in the James Bay drainage basin, begins at Chapleau Lake and is the main tributary of Kapuskasing Lake.
++++++++++++++++++++++++++++++++++++
[{'drainage basin': 'James Bay drainage basin', 'watercourse': 'Chapleau River'}]
++++++++++++++++++++++++++++++++++++



llama_print_timings:        load time =    5714.93 ms
llama_print_timings:      sample time =       5.19 ms /    30 runs   (    0.17 ms per token,  5784.81 tokens per second)
llama_print_timings: prompt eval time =    4182.87 ms /   482 tokens (    8.68 ms per token,   115.23 tokens per second)
llama_print_timings:        eval time =     885.92 ms /    29 runs   (   30.55 ms per token,    32.73 tokens per second)
llama_print_timings:       total time =    5184.37 ms /   511 tokens
 33%|███▎      | 1/3 [00:27<00:55, 27.65s/it]Llama.generate: prefix-match hit


[{'sub': 'Chapleau River', 'rel': 'tributary', 'obj': 'James Bay drainage basin'}]
{'model1': 'llama-2-7b.Q4_0.gguf', 'model2': 'llama-2-7b.Q4_0.gguf', 'sent': 'Chapleau River is in the James Bay drainage basin, begins at Chapleau Lake and is the main tributary of Kapuskasing Lake.', 'triples': [{'sub': 'Chapleau River', 'rel': 'tributary', 'obj': 'James Bay drainage basin'}]}


 67%|██████▋   | 2/3 [00:31<00:13, 13.86s/it]

The mountains classification was won by Nicolas Edet (Cofidis)
++++++++++++++++++++++++++++++++++++
[]
++++++++++++++++++++++++++++++++++++
{'triplets': []}
{'model1': 'llama-2-7b.Q4_0.gguf', 'model2': 'llama-2-7b.Q4_0.gguf', 'sent': 'The mountains classification was won by Nicolas Edet (Cofidis)', 'triples': []}


Llama.generate: prefix-match hit

llama_print_timings:        load time =    5714.93 ms
llama_print_timings:      sample time =      49.25 ms /   256 runs   (    0.19 ms per token,  5198.08 tokens per second)
llama_print_timings: prompt eval time =    4187.05 ms /   472 tokens (    8.87 ms per token,   112.73 tokens per second)
llama_print_timings:        eval time =    9173.66 ms /   255 runs   (   35.98 ms per token,    27.80 tokens per second)
llama_print_timings:       total time =   13965.54 ms /   727 tokens
100%|██████████| 3/3 [00:45<00:00, 15.28s/it]

The wreathed hornbill (Rhyticeros undulatus), also known as the bar-pouched wreathed hornbill, is a species of hornbill found in forests from far north-eastern India and Bhutan, east and south through mainland Southeast Asia and the Greater Sundas in Indonesia, except Sulawesi.
++++++++++++++++++++++++++++++++++++
[{'taxon': 'Rhyticeros undulatus'}]
++++++++++++++++++++++++++++++++++++
{'triplets': []}
{'model1': 'llama-2-7b.Q4_0.gguf', 'model2': 'llama-2-7b.Q4_0.gguf', 'sent': 'The wreathed hornbill (Rhyticeros undulatus), also known as the bar-pouched wreathed hornbill, is a species of hornbill found in forests from far north-eastern India and Bhutan, east and south through mainland Southeast Asia and the Greater Sundas in Indonesia, except Sulawesi.', 'triples': []}


[{'model1': 'llama-2-7b.Q4_0.gguf',
  'model2': 'llama-2-7b.Q4_0.gguf',
  'sent': 'Chapleau River is in the James Bay drainage basin, begins at Chapleau Lake and is the main tributary of Kapuskasing Lake.',
  'triples': [{'sub': 'Chapleau River',
    'rel': 'tributary',
    'obj': 'James Bay drainage basin'}]},
 {'model1': 'llama-2-7b.Q4_0.gguf',
  'model2': 'llama-2-7b.Q4_0.gguf',
  'sent': 'The mountains classification was won by Nicolas Edet (Cofidis)',
  'triples': []},
 {'model1': 'llama-2-7b.Q4_0.gguf',
  'model2': 'llama-2-7b.Q4_0.gguf',
  'sent': 'The wreathed hornbill (Rhyticeros undulatus), also known as the bar-pouched wreathed hornbill, is a species of hornbill found in forests from far north-eastern India and Bhutan, east and south through mainland Southeast Asia and the Greater Sundas in Indonesia, except Sulawesi.',
  'triples': []}]

## Базовая модель

In [11]:
builder = TripletBuilder(relations_schema,
                         n_runs=1000)

builder.load_data(GROUND_TRUTH_PATH, test_index)

generated_triples = builder.get_triplets()

with open(os.path.join('artifacts', DOMAIN, 'triples_base.jsonl'), 'w', encoding='utf-8') as f:
    for entry in generated_triples:
        json_record = json.dumps(entry, ensure_ascii=False)
        f.write(json_record + '\n')

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../models/llama-2-7b.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32             

Chapleau River is in the James Bay drainage basin, begins at Chapleau Lake and is the main tributary of Kapuskasing Lake.
++++++++++++++++++++++++++++++++++++
[{'drainage basin': 'James Bay drainage basin', 'watercourse': 'Chapleau River'}]
++++++++++++++++++++++++++++++++++++



llama_print_timings:        load time =    4136.51 ms
llama_print_timings:      sample time =       5.88 ms /    30 runs   (    0.20 ms per token,  5097.71 tokens per second)
llama_print_timings: prompt eval time =    4123.55 ms /   482 tokens (    8.56 ms per token,   116.89 tokens per second)
llama_print_timings:        eval time =     889.83 ms /    29 runs   (   30.68 ms per token,    32.59 tokens per second)
llama_print_timings:       total time =    5103.16 ms /   511 tokens
  1%|          | 1/157 [00:25<1:06:24, 25.54s/it]Llama.generate: prefix-match hit


[{'sub': 'Chapleau River', 'rel': 'tributary', 'obj': 'James Bay drainage basin'}]
{'model1': 'llama-2-7b.Q4_0.gguf', 'model2': 'llama-2-7b.Q4_0.gguf', 'sent': 'Chapleau River is in the James Bay drainage basin, begins at Chapleau Lake and is the main tributary of Kapuskasing Lake.', 'triples': [{'sub': 'Chapleau River', 'rel': 'tributary', 'obj': 'James Bay drainage basin'}]}



llama_print_timings:        load time =    4136.51 ms
llama_print_timings:      sample time =      43.33 ms /   256 runs   (    0.17 ms per token,  5908.28 tokens per second)
llama_print_timings: prompt eval time =    7544.99 ms /   886 tokens (    8.52 ms per token,   117.43 tokens per second)
llama_print_timings:        eval time =    8217.99 ms /   255 runs   (   32.23 ms per token,    31.03 tokens per second)
llama_print_timings:       total time =   16262.48 ms /  1141 tokens
  1%|▏         | 2/157 [00:41<51:53, 20.09s/it]  Llama.generate: prefix-match hit


...

Lugenda River flows in a south-north direction from Lake Amaramba/Lake Chiuta and is the largest tributary of the Ruvuma River.
++++++++++++++++++++++++++++++++++++
[{'watercourse': 'Lugenda River'}]
++++++++++++++++++++++++++++++++++++
{'triplets': []}
{'model1': 'llama-2-7b.Q4_0.gguf', 'model2': 'llama-2-7b.Q4_0.gguf', 'sent': 'Lugenda River flows in a south-north direction from Lake Amaramba/Lake Chiuta and is the largest tributary of the Ruvuma River.', 'triples': []}



llama_print_timings:        load time =    4136.51 ms
llama_print_timings:      sample time =      42.73 ms /   256 runs   (    0.17 ms per token,  5990.69 tokens per second)
llama_print_timings: prompt eval time =     686.36 ms /    72 tokens (    9.53 ms per token,   104.90 tokens per second)
llama_print_timings:        eval time =    8301.22 ms /   255 runs   (   32.55 ms per token,    30.72 tokens per second)
llama_print_timings:       total time =    9388.33 ms /   327 tokens
 99%|█████████▉| 156/157 [24:59<00:09,  9.49s/it]Llama.generate: prefix-match hit


The Sandanska Bistritsa takes its source from the south-eastern corner of the Tevnoto Lake in Northern Pirin at an altitude of 2,512Â m. It flows in southern direction through another four lakes of the Malokamenishki Lakes group and then turns westwards.
++++++++++++++++++++++++++++++++++++
[{'mountain range': 'Malokamenishki Lakes group'}]
++++++++++++++++++++++++++++++++++++
{'triplets': []}
{'model1': 'llama-2-7b.Q4_0.gguf', 'model2': 'llama-2-7b.Q4_0.gguf', 'sent': 'The Sandanska Bistritsa takes its source from the south-eastern corner of the Tevnoto Lake in Northern Pirin at an altitude of 2,512Â\xa0m. It flows in southern direction through another four lakes of the Malokamenishki Lakes group and then turns westwards.', 'triples': []}



llama_print_timings:        load time =    4136.51 ms
llama_print_timings:      sample time =      43.23 ms /   256 runs   (    0.17 ms per token,  5922.50 tokens per second)
llama_print_timings: prompt eval time =     546.17 ms /    51 tokens (   10.71 ms per token,    93.38 tokens per second)
llama_print_timings:        eval time =    8263.50 ms /   255 runs   (   32.41 ms per token,    30.86 tokens per second)
llama_print_timings:       total time =    9213.61 ms /   306 tokens
100%|██████████| 157/157 [25:08<00:00,  9.61s/it]

The willet (Tringa semipalmata), formerly in the monotypic genus Catoptrophorus as Catoptrophorus semipalmatus, is a large shorebird in the family Scolopacidae.
++++++++++++++++++++++++++++++++++++
[{'taxon': 'Tringa semipalmata'}]
++++++++++++++++++++++++++++++++++++
{'triplets': []}
{'model1': 'llama-2-7b.Q4_0.gguf', 'model2': 'llama-2-7b.Q4_0.gguf', 'sent': 'The willet (Tringa semipalmata), formerly in the monotypic genus Catoptrophorus as Catoptrophorus semipalmatus, is a large shorebird in the family Scolopacidae.', 'triples': []}


## Дообученная модель

In [12]:
builder = TripletBuilder(relations_schema,
                         model1_path=os.path.join('artifacts', DOMAIN, 'model_1/Llama-2-7b-m1ft-q4.gguf'),
                         model2_path=os.path.join('artifacts', DOMAIN, 'model_2/Llama-2-7b-m2ft-q4.gguf'),
                         n_runs=1000)

builder.load_data(GROUND_TRUTH_PATH, test_index)

generated_triples = builder.get_triplets()

with open(os.path.join('artifacts', DOMAIN, 'triples_ft.jsonl'), 'w', encoding='utf-8') as f:
    for entry in generated_triples:
        json_record = json.dumps(entry, ensure_ascii=False)
        f.write(json_record + '\n')

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from artifacts/nature/model_1/Llama-2-7b-m1ft-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 32000
llama_model_loader: - kv   3:                       llama.context_length u32              = 4096
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   7:                 llama.rope.dim

Chapleau River is in the James Bay drainage basin, begins at Chapleau Lake and is the main tributary of Kapuskasing Lake.
++++++++++++++++++++++++++++++++++++
[{'watercourse': 'Chapleau River', 'body_of_water': 'Chapleau Lake'}]
++++++++++++++++++++++++++++++++++++



llama_print_timings:        load time =    4245.28 ms
llama_print_timings:      sample time =       5.40 ms /    33 runs   (    0.16 ms per token,  6106.59 tokens per second)
llama_print_timings: prompt eval time =    8123.06 ms /   480 tokens (   16.92 ms per token,    59.09 tokens per second)
llama_print_timings:        eval time =     973.70 ms /    33 runs   (   29.51 ms per token,    33.89 tokens per second)
llama_print_timings:       total time =    9179.30 ms /   513 tokens
  1%|          | 1/157 [00:22<57:52, 22.26s/it]Llama.generate: prefix-match hit


[{'sub': 'Chapleau River', 'rel': 'origin of the watercourse', 'obj': 'Chapleau Lake'}]
{'model1': 'Llama-2-7b-m1ft-q4.gguf', 'model2': 'Llama-2-7b-m2ft-q4.gguf', 'sent': 'Chapleau River is in the James Bay drainage basin, begins at Chapleau Lake and is the main tributary of Kapuskasing Lake.', 'triples': [{'sub': 'Chapleau River', 'rel': 'origin of the watercourse', 'obj': 'Chapleau Lake'}]}



llama_print_timings:        load time =    4202.89 ms
llama_print_timings:      sample time =       3.84 ms /    24 runs   (    0.16 ms per token,  6256.52 tokens per second)
llama_print_timings: prompt eval time =    3277.39 ms /    18 tokens (  182.08 ms per token,     5.49 tokens per second)
llama_print_timings:        eval time =     734.95 ms /    23 runs   (   31.95 ms per token,    31.29 tokens per second)
llama_print_timings:       total time =    4049.89 ms /    41 tokens
  1%|▏         | 2/157 [00:26<29:50, 11.55s/it]Llama.generate: prefix-match hit


...

The Sandanska Bistritsa takes its source from the south-eastern corner of the Tevnoto Lake in Northern Pirin at an altitude of 2,512Â m. It flows in southern direction through another four lakes of the Malokamenishki Lakes group and then turns westwards.
++++++++++++++++++++++++++++++++++++
[{'watercourse': 'Tevnoto Lake'}]
++++++++++++++++++++++++++++++++++++
{'triplets': []}
{'model1': 'Llama-2-7b-m1ft-q4.gguf', 'model2': 'Llama-2-7b-m2ft-q4.gguf', 'sent': 'The Sandanska Bistritsa takes its source from the south-eastern corner of the Tevnoto Lake in Northern Pirin at an altitude of 2,512Â\xa0m. It flows in southern direction through another four lakes of the Malokamenishki Lakes group and then turns westwards.', 'triples': []}



llama_print_timings:        load time =    4202.89 ms
llama_print_timings:      sample time =       7.39 ms /    36 runs   (    0.21 ms per token,  4872.77 tokens per second)
llama_print_timings: prompt eval time =     546.41 ms /    51 tokens (   10.71 ms per token,    93.34 tokens per second)
llama_print_timings:        eval time =    1145.04 ms /    35 runs   (   32.72 ms per token,    30.57 tokens per second)
llama_print_timings:       total time =    1774.83 ms /    86 tokens
Llama.generate: prefix-match hit


The willet (Tringa semipalmata), formerly in the monotypic genus Catoptrophorus as Catoptrophorus semipalmatus, is a large shorebird in the family Scolopacidae.
++++++++++++++++++++++++++++++++++++
[{'taxon': 'Willet', 'synonym': 'Catoptrophorus semipalmatus'}]
++++++++++++++++++++++++++++++++++++



llama_print_timings:        load time =    4245.28 ms
llama_print_timings:      sample time =       5.74 ms /    33 runs   (    0.17 ms per token,  5753.14 tokens per second)
llama_print_timings: prompt eval time =    1979.98 ms /   199 tokens (    9.95 ms per token,   100.51 tokens per second)
llama_print_timings:        eval time =     956.60 ms /    32 runs   (   29.89 ms per token,    33.45 tokens per second)
llama_print_timings:       total time =    3022.69 ms /   231 tokens
100%|██████████| 157/157 [08:49<00:00,  3.37s/it]

[{'sub': 'Willet', 'rel': 'taxon synonym', 'obj': 'Catoptrophorus semipalmatus'}]
{'model1': 'Llama-2-7b-m1ft-q4.gguf', 'model2': 'Llama-2-7b-m2ft-q4.gguf', 'sent': 'The willet (Tringa semipalmata), formerly in the monotypic genus Catoptrophorus as Catoptrophorus semipalmatus, is a large shorebird in the family Scolopacidae.', 'triples': [{'sub': 'Willet', 'rel': 'taxon synonym', 'obj': 'Catoptrophorus semipalmatus'}]}


## Создание файла по триплетам Ground Truth

In [13]:
with open(GROUND_TRUTH_PATH, 'r') as f:
    data = f.readlines()

data = [data[i] for i in test_index]

with open(os.path.join('artifacts', DOMAIN, 'triples_gt.jsonl'), 'w', encoding='utf-8') as f:
    for x in data:
        f.write(x)

In [14]:
# Конвертация старого запуска 

# with open('artifacts/movie/mixtral_movie_full_ft.jsonl', 'r') as f:
#     data = f.readlines()

# data = [data[i] for i in test_index]

# with open(os.path.join('artifacts', DOMAIN, 'triples_ft.jsonl'), 'w', encoding='utf-8') as f:
#     for x in data:
#         f.write(x)

# Получение метрик по триплетам

Метрика Жаккарда - отношение пересечения и объединению множеств триплетов первой и второй модели. Триплеты считваются одинаковыми только при полном совпадении всех трех полей

In [15]:
class Jaccard_metric:

    def clean_list(self, A):
        AA = []
        if A is not None:
            for a in A:
                if isinstance(a, dict) and 'sub' in a.keys() and 'rel' in a.keys() and 'obj' in a.keys():
                    AA.append(a)

        return AA


    def count_matching(self, A, B):
        """
        Функция подсчитывает количество одинаковых триплетов
        """

        count = 0
        A = self.clean_list(A)
        B = self.clean_list(B)

        for i in range(len(A)):
            a = A[i]
            for j in range(len(B)):
                b = B[j]
                if a['sub'] == b['sub'] and a['rel'] == b['rel'] and a['obj'] == b['obj']:
                    count += 1
                    continue

        return count


    def count_unique(self, A, B):
        """
        Функция, которая подсчитывает количество уникальных триплетов
        """

        A = self.clean_list(A)
        B = self.clean_list(B)

        return len(A) + len(B) - self.count_matching(A, B)


    def get_metric(self, expected: List[Set], predicted: List[Set]) -> float:

        try:
            X = expected
            Y = predicted
            return self.count_matching(X, Y) / self.count_unique(X, Y)
        except:
            return 0
        
    
    def read_triples(self, path: str) -> List[List[Set]]:
        with open(path, 'r') as f:
            data = f.readlines()

        result = []
        for i_text in data:
            i_text = json.loads(i_text)

            triples = i_text['triples']
            result.append(triples)

        return result
    

    def model_metric(self, expected_path: str, predicted_path: str) -> float:
        """Возвращает среднее значение меры Жаккара по двум наборам триплетов. 
        Триплеты считаются одинаковыми при полном совпадении"""

        expected_list = self.read_triples(expected_path)
        predicted_list = self.read_triples(predicted_path)
        metric_arr = []

        for expected, predicted in zip(expected_list, predicted_list):
            metric = self.get_metric(expected, predicted)
            metric_arr.append(metric)

        return np.array(metric_arr).mean()


In [16]:
jaccard_metric = Jaccard_metric()
jaccard_metric_base = jaccard_metric.model_metric(
    os.path.join('artifacts', DOMAIN, 'triples_gt.jsonl'),
    os.path.join('artifacts', DOMAIN, 'triples_base.jsonl'),
)
jaccard_metric_ft = jaccard_metric.model_metric(
    os.path.join('artifacts', DOMAIN, 'triples_gt.jsonl'),
    os.path.join('artifacts', DOMAIN, 'triples_ft.jsonl'),
)

jaccard_metric_base, jaccard_metric_ft

(0.0, 0.24699221514508135)

Адаптированные классические метрики точности, полноты и f1

In [ ]:
class GraphEvaluator:
    def __init__(self):
        pass

    def get_metrics(self, gold: Set, pred: Set) -> float:
        """
        Method to calculate precision, recall and f1:
            Precision is calculated as correct_triples/predicted_triples and
            Recall as correct_triples/gold_triples
            F1 as the harmonic mean of precision and recall.
        :param gold: items in the gold standard
        :param pred: items in the system prediction
        :return:
            p: float - precision
            r: float - recall
            f1: float - F1
        """
        if len(pred) == 0:
            return 0, 0, 0
        p = len(gold.intersection(pred)) / len(pred)
        r = len(gold.intersection(pred)) / len(gold)
        if p + r > 0:
            f1 = 2 * ((p * r) / (p + r))
        else:
            f1 = 0
        return p, r, f1
    

    def normalize_triple(self, sub_label: str, rel_label: str, obj_label: str) -> str:
        """
        Normalize triples for comparison in precision, recall calculations
        :param sub_label: subject string
        :param rel_label: relation string
        :param obj_label: object string
        :return: a normalized triple as a single concatenated string
        """
        # remove spaces and underscores and make lower case
        sub_label = re.sub(r"(_|\s+)", '', sub_label).lower()
        rel_label = re.sub(r"(_|\s+)", '', rel_label).lower()
        obj_label = re.sub(r"(_|\s+)", '', obj_label).lower()
        # concatenate them to a single string
        tr_key = f"{sub_label}{rel_label}{obj_label}"
        return tr_key
    
    def filter_triplets(self, file_path, pred=False) -> List[str]:
        """
        Filter triplets from a file.
        :param file_path: path to the file containing triplets
        :param pred: flag indicating whether the file contains predictions or gold standard triplets
        :return: filtered triplets as a list, or filtered triplets and model name as a tuple if pred=True
        """
        with open(file_path, 'r') as f:
            data = f.readlines()
        filtered_triplets = []
        ############################################
        # здесь можно указать срез по данным
        ############################################
        for i_line in data:  #[:10]: # срез по данным. Кол-во n_run из тетрадки по генерации триплетов
            json_data = json.loads(i_line)
            for i_triple in json_data['triples']:
                try:
                    if i_triple['rel'] not in ["cost", "main subject", "publication date"]:
                        if i_triple['obj'] != "":
                            filtered_triplets.append(i_triple)
                except:
                    continue

        if pred:
            model = json_data.get('model', f"{json_data.get('model1', '')}+{json_data.get('model2', '')}")
            return [filtered_triplets, model]
        
        return [filtered_triplets]


    def evaluate(self, pred_file_path: str, gold_file_path: str) -> Dict:
        """
        Evaluate the performance of a prediction file against a gold standard file.
        :param pred_file_path: path to the prediction file
        :param gold_file_path: path to the gold standard file
        :return: a dictionary containing the model name, precision, recall, and F1 score
        """
        pred_temp = self.filter_triplets(pred_file_path, pred=True)
        pred = pred_temp[0]
        model = pred_temp[1]
        gold = self.filter_triplets(gold_file_path)[0]
        pred_ready = {self.normalize_triple(tr['sub'], tr['rel'], tr['obj']) for tr in pred}
        gold_ready = {self.normalize_triple(tr['sub'], tr['rel'], tr['obj']) for tr in gold}
        p, r, f1 = self.get_metrics(gold_ready, pred_ready)
        return {"model": model, "precision": p, "recall": r, "f1": f1}

In [18]:
evaluator = GraphEvaluator()

metrics_base = evaluator.evaluate(
    os.path.join('artifacts', DOMAIN, 'triples_base.jsonl'), 
    os.path.join('artifacts', DOMAIN, 'triples_gt.jsonl'))

metrics_ft = evaluator.evaluate(
    os.path.join('artifacts', DOMAIN, 'triples_ft.jsonl'), 
    os.path.join('artifacts', DOMAIN, 'triples_gt.jsonl'))

metrics_base, metrics_ft

({'model': 'llama-2-7b.Q4_0.gguf+llama-2-7b.Q4_0.gguf',
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0},
 {'model': 'Llama-2-7b-m1ft-q4.gguf+Llama-2-7b-m2ft-q4.gguf',
  'precision': 0.5555555555555556,
  'recall': 0.28125,
  'f1': 0.37344398340248963})

Сохранение

In [19]:
metrics_df = pd.DataFrame({
    'domain': [DOMAIN, DOMAIN, DOMAIN],
    'knowknowledge_graph': ['ground truth','base', 'finetuned'],
    'jaccard': [1.0, jaccard_metric_base, jaccard_metric_ft],
    'precision': [1.0, metrics_base['precision'], metrics_ft['precision']],
    'recall': [1.0, metrics_base['recall'], metrics_ft['recall']],
    'f1': [1.0, metrics_base['f1'], metrics_ft['f1']],
    })

fn = os.path.join('artifacts', DOMAIN, 'kg_metrics.csv')
metrics_df.to_csv(fn, index=False)

pd.read_csv(fn)


,domain,knowknowledge_graph,jaccard,precision,recall,f1
0,nature,ground truth,1.000000,1.000000,1.00000,1.000000
1,nature,base,0.000000,0.000000,0.00000,0.000000
2,nature,finetuned,0.246992,0.555556,0.28125,0.373444
